<a href="https://colab.research.google.com/github/Hassan-Irfan-bot/Symptobot/blob/main/SymtoBot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

panda is sued to read and perform operation on tabular data.
numpy to apply all arethmatic operations on data

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Symtobot/Training.csv')
data.head(5)

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [5]:
headings = data.columns.tolist()
# headings

In [6]:
# dropped prognosis coloumn and stored all remaining coloumn to X
X = data.drop('prognosis',axis=1)
Y = data['prognosis']
#to get rid of headings
x = X.to_numpy()
y = Y.to_numpy()
encoder = LabelEncoder()
y = encoder.fit_transform(Y)
print(x)
print(y)

[[1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 1 1 1]]
[15 15 15 ... 38 35 27]


In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 0.2 means 20 perc of data is for test rest is for training

In [8]:
print(f"shape of X_train{X_train.shape} \n Shape of Y_train{Y_train.shape} \n X_test_shape {X_test.shape} \n Y_test.shape{Y_test.shape}")

shape of X_train(3936, 132) 
 Shape of Y_train(3936,) 
 X_test_shape (984, 132) 
 Y_test.shape(984,)


In [9]:
X_train[0]    #first entery of data

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
max_sequence_length = X.shape[1]
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')


In [11]:
# NLP => Sequential model
model = Sequential([
    Embedding(input_dim=2, output_dim=64, input_length=max_sequence_length),
    LSTM(units=64, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dense(units=len(encoder.classes_), activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 132, 64)           128       
                                                                 
 lstm (LSTM)                 (None, 132, 64)           33024     
                                                                 
 dropout (Dropout)           (None, 132, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 41)                2665      
                                                                 
Total params: 68841 (268.91 KB)
Trainable params: 68841 (268.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
123/123 [==============================] - 23s 151ms/step - loss: 3.3172 - accuracy: 0.1159
Epoch 2/10
123/123 [==============================] - 18s 144ms/step - loss: 1.5083 - accuracy: 0.5742
Epoch 3/10
123/123 [==============================] - 17s 137ms/step - loss: 0.8168 - accuracy: 0.8369
Epoch 4/10
123/123 [==============================] - 17s 140ms/step - loss: 0.5462 - accuracy: 0.8885
Epoch 5/10
123/123 [==============================] - 17s 141ms/step - loss: 0.3590 - accuracy: 0.9271
Epoch 6/10
123/123 [==============================] - 19s 156ms/step - loss: 0.3030 - accuracy: 0.9309
Epoch 7/10
123/123 [==============================] - 18s 145ms/step - loss: 0.2148 - accuracy: 0.9568
Epoch 8/10
123/123 [==============================] - 17s 139ms/step - loss: 0.1703 - accuracy: 0.9670
Epoch 9/10
123/123 [==============================] - 17s 137ms/step - loss: 0.1731 - accuracy: 0.9609
Epoch 10/10
123/123 [==============================] - 17s 142ms/step - l

In [15]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

31/31 [==============================] - 2s 41ms/step - loss: 0.1382 - accuracy: 0.9573
Test Loss: 0.1382269412279129
Test Accuracy: 0.957317054271698


In [22]:
model.save('/content/drive/MyDrive/Symtobot/data.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
temp = data.drop('prognosis', axis = 1)
headings = temp.columns.tolist()

In [18]:
def preprocess_input(input_symptoms, all_symptoms):
    symptom_to_index = {symptom: i for i, symptom in enumerate(all_symptoms)}

    input_array = np.zeros(len(all_symptoms))

    for symptom in input_symptoms:
        if symptom in symptom_to_index:
            input_array[symptom_to_index[symptom]] = 1

    return input_array

all_symptoms = headings
input_symptoms = ['continuous_sneezing']
input_array = preprocess_input(input_symptoms, all_symptoms)
print(input_array.shape)

(132,)


In [19]:
input_array_reshaped = input_array.reshape(1, -1)

predictions = model.predict(input_array_reshaped)

print(predictions)


1/1 [==============================] - 1s 1s/step
[[2.2558512e-05 5.4804748e-04 6.0967031e-05 1.8879358e-05 1.2309644e-01
  3.8435552e-05 1.5592452e-02 5.4908439e-04 1.4385401e-04 2.7048411e-02
  1.1894123e-05 1.0288124e-04 1.3932368e-03 6.7852513e-04 7.4580573e-02
  3.8229775e-01 1.6969938e-02 1.2925216e-03 3.3251005e-01 6.7212892e-04
  7.7042254e-03 1.3046258e-03 1.1984296e-04 8.7860069e-04 1.1705675e-04
  3.0125200e-04 2.6986509e-04 5.6955134e-03 4.4455525e-04 2.4544960e-04
  1.0441376e-03 3.6019741e-05 2.4736996e-04 1.0985060e-04 3.2154695e-04
  3.2945503e-05 2.8678402e-03 6.1868709e-06 5.0464022e-04 4.0866580e-05
  7.8895544e-05]]


In [20]:
def get_predicted_symptom(pred_index, symptom_names = all_symptoms):
    return symptom_names[pred_index]

In [21]:
predicted_index = np.argmax(predictions)
predicted_symptom = get_predicted_symptom(predicted_index, all_symptoms)
print("Predicted Symptom:", predicted_symptom)

Predicted Symptom: weight_gain


In [24]:
pip install fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import numpy as np

app = FastAPI()

# Load the trained model
model = load_model("/content/drive/MyDrive/Symtobot/data.h5")
encoder = LabelEncoder()

class SymptomInput(BaseModel):
    symptoms: list

@app.post("/predict")
async def predict(symptom_input: SymptomInput):
    try:
        # Preprocess input symptoms
        input_array = preprocess_input(symptom_input.symptoms, headings)

        # Predict disease
        prediction = model.predict(input_array.reshape(1, -1))
        predicted_index = np.argmax(prediction)
        predicted_disease = get_predicted_symptom(predicted_index, encoder.classes_)

        return {"predicted_disease": predicted_disease}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
